In [1]:
import pandas as pd
import glob
import ipywidgets as wid
import sys
import itertools
sys.path.append('../')
import utils.files_utils
import os

In [2]:
AUDIO_DIR = utils.files_utils.get_audios_path('brmf_4b')
EVALUATIONS_PATH = 'evaluations.csv'

In [92]:
audios = glob.glob(os.path.join(utils.files_utils.get_audios_path('brmf_4b'), '*_to_*.mp3'))

def group_to_records(p):
    basename = os.path.splitext(os.path.basename(p))[0]
    s = basename.split('_')
    return {
        'titulo': s[0],
        'autor': s[1],
        'target': s[3],
        'id': s[4],
        'audio': p
    }

data = pd.DataFrame.from_records([
   group_to_records(p) for p in audios 
])

In [154]:
def transfered_to_original(titulo):
    p = glob.glob(os.path.join(AUDIO_DIR, titulo + '_orig*.mp3'))
    assert(len(p) == 1)
    return p[0]

def original_to_image(f: str):
    base = os.path.dirname(f)
    file = os.path.basename(f)
    t, r = file.split('_', maxsplit=1)
    r = os.path.splitext(r)[0]
    # TODO: borrar cuando no haya mas ID
    r = r.rsplit('_', maxsplit=1)[0]
    return f'{base}/{t}.mid_{r}.pdf.png'

In [164]:
def setup(ds):
    skip_answered = wid.Checkbox(description="Skip answered:", value=True)
    start_btn = wid.Button(description="Start", disabled=False)
    evaluator = wid.Text(description="Evaluator")
    
    start_buttons = wid.HBox([skip_answered, evaluator, start_btn])

    song_name = wid.Label('--')
    orig_label = wid.Label(f'Original (--):')
    orig_audio = wid.Audio(autoplay=False, loop=False)
    orig_image = wid.Image()
    trans_label = wid.Label(f'Transfered (--):')
    trans_audio = wid.Audio(autoplay=False, loop=False)    
    trans_image = wid.Image()
    
    audios = wid.HBox([
        wid.VBox([orig_label, orig_audio, orig_image]),
        wid.VBox([trans_label, trans_audio, trans_image])
    ])
    #sheets = wid.HBox([
    #   wid.Image(value=transfered_to_original(titulo).replace('mp3', 'png')) 
    #])
    mus_slider = wid.IntSlider(description='Musicalidad?:', min=1, max=5, value=3, style={'description_width': '120px'}, disabled=True)
    new_slider = wid.IntSlider(description='Nuevo estilo?:', min=1, max=5, value=3, style={'description_width': '120px'}, disabled=True)
    old_slider = wid.IntSlider(description='Respeta original?:', min=1, max=5, value=3, style={'description_width': '120px'}, disabled=True)
       
    ratings = wid.VBox([mus_slider, new_slider, old_slider])
    
    save_btn = wid.Button(description="Save", disabled=True)
    next_btn = wid.Button(description="Next", disabled=True)
    save_btns = wid.HBox([next_btn, save_btn])
    
    i = -1
    resps = []
    
    def start(evt):
        nonlocal ds
        skip_answered.disabled = True
        start_btn.disabled = True
        evaluator.disabled = True
        
        mus_slider.disabled = False
        new_slider.disabled = False
        old_slider.disabled = False
        
        save_btn.disabled = False
        next_btn.disabled = False
        
        if skip_answered.value and os.path.isfile(EVALUATIONS_PATH):
            answered = pd.read_csv(EVALUATIONS_PATH)['path']
            ds = [x for x in ds if x['audio'] not in answered]
        
        cont(evt)
        
    def cont(evt):
        nonlocal i, resps
        i += 1
        resps.append(dict(
            path=ds[i]['audio'],
            evaluator=evaluator.value,
            mus=mus_slider.value,
            new=new_slider.value,
            old=old_slider.value
        ))
        
        orig_audio_fn = transfered_to_original(ds[i]['titulo'])
        orig_image_fn = original_to_image(orig_audio_fn)
        trans_audio_fn = ds[i]['audio']
        trans_image_fn = original_to_image(trans_audio_fn)
        
        for x in [orig_audio_fn, trans_audio_fn, ori]:
            if not os.path.isfile(x):
                print(f'Could not find {x}')
        
        autor = ds[i]['autor']
        dest = ds[i]['target']
        titulo = ds[i]['titulo']
        
        song_name.value = titulo
        orig_label.value = f'Original ({autor}):'
        trans_label.value = f'Transfered ({dest}):'
        
        orig_audio.set_value_from_file(orig_audio_fn)
        trans_audio.set_value_from_file(trans_audio_fn)
        
        orig_image.set_value_from_file(orig_image_fn)
        trans_image.set_value_from_file(trans_image_fn)
        
        
        mus_slider.value = 3
        new_slider.value = 3
        old_slider.value = 3
        
    def save(evt):
        if os.path.isfile(EVALUATIONS_PATH):
            resps_df = pd.read_csv(EVALUATIONS_PATH)
        else:
            resps_df = pd.DataFrame()
        pd.concat([resps_df, pd.DataFrame.from_records(resps)]).to_csv(EVALUATIONS_PATH)
        next_btn.disabled = True
        save_btn.disabled = True
        
    start_btn.on_click(start)
    next_btn.on_click(cont)
    save_btn.on_click(save)
    
    display(wid.VBox([start_buttons, song_name, audios, ratings, save_btns]))

In [165]:
shuffled_data = data.sample(frac=1)

g = [(k, v.iterrows()) for k, v in list(shuffled_data.groupby(['autor', 'target']))]

mixed_data = []
cycle = itertools.cycle(g)
while len(mixed_data) < shuffled_data.shape[0]:
    try:
        n = next(cycle)
        mixed_data.append(next(n[1]))
    except StopIteration:
        pass

In [166]:
setup([x[1] for x in mixed_data])

FileNotFoundError: [Errno 2] No such file or directory: '/home/mmiguel/style-transfer/data/Audios/brmf_4b/canzoni04.mid_Frescobaldi_to_Bach.pdf.png'

FileNotFoundError: [Errno 2] No such file or directory: '/home/mmiguel/style-transfer/data/Audios/brmf_4b/canzoni33.mid_Frescobaldi_to_Mozart.pdf.png'

In [196]:
audio = wid.Audio(autoplay=False)
audio.set_value_from_file(data.iloc[0]['audio'])
audio

Audio(value=b'ID3\x04\x00\x00\x00\x00\x00#TSSE\x00\x00\x00\x0f\x00\x00\x03Lavf58.29.100\x00\x00\x00\x00\x00\x0…

In [30]:
cat resps.csv

,mus,new,old
0,2,4,2
1,2,5,4
